In [14]:
import torch

In [15]:
from models.classifier import TransformerEncoder

In [16]:
hparams = {
    "hidden_size": 48,  # size of the hidden layers and embeddings
    "hidden_ff": 96,  # size of the position-wise feed-forward layer
    "n_encoders": 4,  # number of encoder blocks
    "n_heads": 2,  # number of attention heads in the multiheadattention module
    "n_local": 2,  # number of local attention heads
    "local_window_size": 4,  # size of the window for local attention
    'batch_size': 4,
    "max_length": 30,  # maximum length of the input sequence
    "vocab_size": 100,  # size of the vocabulary
    "learning_rate": 0.001,
    "num_epochs": 30,
    "attention_type": "performer",
    "norm_type": "rezero",
    "num_random_features": 32,  # number of random features for the Attention module (Performer uses this)
    "emb_dropout": 0.1,  # dropout for the embedding block
    "fw_dropout": 0.1,  # dropout for the position-wise feed-forward layer
    "att_dropout": 0.1,  # dropout for the multiheadattention module
    "dc_dropout": 0.1,  # dropout for the decoder block
    "hidden_act": "swish",  # activation function for the hidden layers (attention layers use ReLU)
    "epsilon": 1e-8,
    "weight_decay": 0.01,
    "beta1": 0.9,
    "beta2": 0.999,
}
model = TransformerEncoder(hparams)

# Sanity Check
Check if returns values and if the output looks OK.

In [17]:
batch = {
    'tokens': torch.randint(0, model.hparams.vocab_size, (model.hparams.batch_size, model.hparams.max_length)),
    'abspos': torch.arange(0, model.hparams.max_length).unsqueeze(0).repeat(model.hparams.batch_size, 1),
    'age': torch.randint(0, 100, (model.hparams.batch_size, 1)).repeat(1, model.hparams.max_length),
    'padding_mask': torch.zeros(model.hparams.batch_size, model.hparams.max_length),
    'targets': torch.randint(0, 2, (model.hparams.batch_size, 1)).float(),
}

In [18]:
batch

{'tokens': tensor([[43, 65, 24, 97, 45, 59, 54, 47, 62, 62,  9, 32, 74, 80, 81, 48, 12,  9,
          41, 88, 87, 65,  5, 22, 29, 59, 27, 52, 32, 19],
         [53, 72, 38, 38, 87, 45,  7, 33, 87, 38, 83, 69, 62, 59, 28,  5, 56, 28,
          35, 76, 71, 16, 77, 70, 30, 24, 70, 89, 79, 88],
         [14, 32, 58, 69, 44, 72, 23, 21, 99,  0, 65, 70, 13, 26, 39, 28, 42, 41,
          75, 91, 13, 33, 90, 43,  2,  0, 29, 83, 91, 41],
         [13, 82, 68, 52, 77, 58,  9, 33, 34,  3, 65, 30, 13, 10, 85, 24, 99, 62,
          28, 54, 26, 81, 57,  6, 86, 38,  5, 72, 14,  4]]),
 'abspos': tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 

In [19]:
model(batch)

{'logits': tensor([[0.7087],
         [0.6428],
         [0.5000],
         [0.9792]], grad_fn=<AddmmBackward0>),
 'preds': tensor([[0.6701],
         [0.6554],
         [0.6225],
         [0.7269]], grad_fn=<SigmoidBackward0>)}

In [20]:
model.training_step(batch, 0)

/opt/anaconda3/envs/prefer/lib/python3.11/site-packages/pytorch_lightning/core/module.py:447: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(0.7542, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [21]:
model.on_train_epoch_end()

### Full pipeline would be something like

In [22]:
from dataloaders.synthetic import SyntheticDataModule

dataloader = SyntheticDataModule(num_samples=1000, max_length=hparams['max_length'],
                                  batch_size=hparams['batch_size'], vocab_size=hparams['vocab_size'])

In [30]:
next(iter(dataloader.train_dataloader()))

{'tokens': tensor([[27, 78, 37, 49, 41, 84, 38, 72, 64, 49, 39, 34, 17,  6, 90, 88, 58, 69,
          89, 48, 97, 63, 11, 50, 58, 10, 57, 36, 65, 66],
         [17,  5, 16, 16, 38, 95, 88,  7, 30, 34, 70, 56, 35, 22, 90,  7, 41, 33,
          22, 49, 46, 60, 12, 30, 63, 20, 89, 71, 82, 40],
         [29, 43, 78, 73, 37, 55, 72, 40, 30, 51, 94, 28, 99,  8, 36, 74, 78, 15,
          57, 49, 97, 65,  8, 20, 50, 11, 43, 20, 59, 20],
         [91, 38, 14, 88, 43, 96, 33, 28, 17, 18, 94, 79, 89, 62, 86, 38, 49,  0,
          84,  0, 10, 43, 49, 84, 61, 45, 26, 47, 59, 73]]),
 'abspos': tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 

# Import test data

In [ ]:
from dataset import DataModule
sequence_path = 'fake_data/sequence_data.parquet'
targets_path = 'fake_data/targets.csv'
vocab_path = 'fake_data/vocab.json'

datamodule = DataModule(sequence_path, 2, targets_path, vocab_path, subset=True)
datamodule.setup()
loader = datamodule.train_dataloader()

next(iter(loader))

 Accesssing element 1
 Person ID: 3
 Accesssing element 0
 Person ID: 1


RuntimeError: each element in list of batch should be of equal size

In [25]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning import Trainer
#from pytorch_lightning.loggers import TensorBoardLogger

model_checkpoint = ModelCheckpoint(monitor='val/ap', save_top_k=2, save_last=True, mode='max')
early_stopping = EarlyStopping(monitor='val/ap', patience=5, mode='max')
lr_monitor = LearningRateMonitor(logging_interval='step')
#logger = TensorBoardLogger("lightning_logs", name="cls_logs")

trainer = Trainer(max_epochs=30,
                accelerator="cpu",   ### change to "cuda" or "gpu" or 'msp'
                limit_train_batches=0.5,
                #logger=logger,
                accumulate_grad_batches=4,
                num_sanity_val_steps=8,
                callbacks = [model_checkpoint, early_stopping, lr_monitor],
                check_val_every_n_epoch=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [26]:
trainer.fit(model, dataloader)

Steps per epoch: 100

   | Name        | Type                   | Params | Mode 
----------------------------------------------------------------
0  | transformer | Transformer            | 79.5 K | train
1  | decoder     | CLS_Decoder            | 2.4 K  | train
2  | loss        | BCEWithLogitsLoss      | 0      | train
3  | train_loss  | MeanMetric             | 0      | train
4  | val_loss    | MeanMetric             | 0      | train
5  | test_loss   | MeanMetric             | 0      | train
6  | train_acc   | BinaryAccuracy         | 0      | train
7  | val_acc     | BinaryAccuracy         | 0      | train
8  | test_acc    | BinaryAccuracy         | 0      | train
9  | train_mcc   | BinaryMatthewsCorrCoef | 0      | train
10 | val_mcc     | BinaryMatthewsCorrCoef | 0      | train
11 | test_mcc    | BinaryMatthewsCorrCoef | 0      | train
12 | train_ap    | BinaryAveragePrecision | 0      | train
13 | val_ap      | BinaryAveragePrecision | 0      | train
14 | test_ap     | BinaryAve

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/envs/prefer/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/opt/anaconda3/envs/prefer/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples found in target, recall is undefined. Setting recall to one for all thresholds.
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/anaconda3/envs/prefer/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/envs/prefer/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
